# Multiclass SVM 구현

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test) #input데이터 스케일링 

In [4]:
X_train

,sepal_length,sepal_width,petal_length,petal_width
110,6.5,3.2,5.1,2.0
69,5.6,2.5,3.9,1.1
148,6.2,3.4,5.4,2.3
39,5.1,3.4,1.5,0.2
53,5.5,2.3,4.0,1.3
...,...,...,...,...
64,5.6,2.9,3.6,1.3
91,6.1,3.0,4.6,1.4
81,5.5,2.4,3.7,1.0
51,6.4,3.2,4.5,1.5


In [5]:
X_test

,sepal_length,sepal_width,petal_length,petal_width
96,5.7,2.9,4.2,1.3
73,6.1,2.8,4.7,1.2
134,6.1,2.6,5.6,1.4
41,4.5,2.3,1.3,0.3
70,5.9,3.2,4.8,1.8
116,6.5,3.0,5.5,1.8
19,5.1,3.8,1.5,0.3
138,6.0,3.0,4.8,1.8
33,5.5,4.2,1.4,0.2
89,5.5,2.5,4.0,1.3


In [6]:
#일단 y데이터를 원핫인코딩해주자. 
y_train = pd.get_dummies(y_train)

In [7]:
y_train 
#각 라벨에 해당하면 1의값을 가짐

,setosa,versicolor,virginica
110,0,0,1
69,0,1,0
148,0,0,1
39,1,0,0
53,0,1,0
...,...,...,...
64,0,1,0
91,0,1,0
81,0,1,0
51,0,1,0


In [8]:
#일대일로 분류해주는 svm3개 생성. 각각의 라벨이 맞는지 아닌지 판별해주는 분류기
#각 파라미터는 svm기본값으로 설정하였다. 
svm1 = SVC(kernel ='rbf', C = 1, gamma = 'scale')
svm2 = SVC(kernel ='rbf', C = 1, gamma = 'scale')
svm3 = SVC(kernel ='rbf', C = 1, gamma = 'scale')

In [10]:
#각각 라벨별로 fit을 진행
svm1.fit(X_train,y_train.iloc[:,0]) #setosa
svm2.fit(X_train,y_train.iloc[:,1]) #versicolor
svm3.fit(X_train,y_train.iloc[:,2]) #virginica

SVC(C=1)

In [11]:
#한개의 svm에 대해서 테스트해보자. 
pred=svm1.predict(X_test)
print(pred)
print(svm1.decision_function(X_test)) #각 값과 초평면사이의 거리를 구하는 method
#거리의 값이 음수이면 0, 양수이면 1으로 분류함

[0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.32258826 -1.4975558  -1.62724425  0.80861395 -1.4818528  -1.5369139
  1.2331735  -1.54096736  1.32863141 -1.35461015 -1.52658645  1.2240527
  0.98769165 -1.58277702 -1.54189369 -1.3914532   1.10964084 -1.37139836
 -1.62931678 -1.39983345  1.0433074  -1.54093921 -1.49360698 -1.40219071
 -1.38386062  1.26637488  1.04871975 -1.43255332 -1.56520934 -1.23675972]


- 각 분류기 svm1,2,3에서 만들어진 pred값에서 voting을 진행하여 많은 개수의 쪽으로 진행하면 됨 
- 동점이 발생할 경우 : 라벨의 개수가 홀수이면 모두 같은 값을 보이는 경우/라벨의 개수가 짝수이면 반반씩 같은 값을 가지는 경우 
- voting을 통해서는 너무 많은 변수가 생길 수 있음 
- pred값이 아닌 decision_function의 값을 이용하여 비교하는 것이 나음..
- 절댓값이 가장 큰 것으로 분류결정이 되지 않을까...라고 생각했는데 정확도 0나와서 그냥 가장 큰 숫자로 변경..

In [27]:
#각 svm별 decision_function배열을 구하고 이어붙인다음, 값이 가장 큰 라벨을 고르는 함수를 만들어보자. 
def one_rest(models,data,label):
    dtc=None #배열을 담을 빈 배열 생성 
    for model in models: #models는 각각의 분류기가 담길 리스트임
        if dtc is None: #처음 담아지는 경우
            dtc=model.decision_function(data) #거리배열 구하기 
        else:
            dtc=np.vstack((dtc,model.decision_function(data)))#세로로 이어붙이기 
    dtc=dtc.T #transpose를 해야 y_train처럼 X_data의 길이만큼의 행길이와 label개수만큼의 열이 생성됨 
    result=[] #값이 젤 큰 라벨을 담을 리스트 
    for pred in dtc:
        
        result.append(label[pred.argmax()])
    print('prediction:',result)
    return result 


In [28]:
models=[svm1,svm2,svm3]
label=['setosa','versicolor','virginica']
prediction=one_rest(models,X_test,label)
accuracy_score(y_test,prediction)

prediction: ['versicolor', 'versicolor', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'versicolor', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'versicolor', 'virginica', 'setosa', 'virginica', 'virginica', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.9